In [1]:
import lucene

lucene.initVM()

In [2]:
# !pip install pandas tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 7.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.3/502.3 kB 13.6 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 11.0 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip available: 22.3.1 -> 23.1.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import pandas as pd
import re
from tqdm import tqdm
import ast

In [4]:
bad_words = pd.read_csv('/Project/data/bad_word.txt', header=None)
# bad_words = pd.read_csv('data/bad_word.txt', header=None)
bad_words_set = set(bad_words[0].values)

In [5]:
bad_words_list= bad_words.values.flatten()

In [6]:
news_dataset = pd.read_csv('/Project/data/pos_tags_dataset_bw.csv',header=0)
# news_dataset = pd.read_csv('data/pos_tags_dataset.csv',header=0)
# labels = pd.read_csv('data/comments_model1_outputs.txt',header=None)

In [9]:
news_dataset.head()

,index,comment,cleaned_comment,pos_tag
0,0,A lot of the clients I work with use SVB. This...,a lot of the clients i work with use svb this ...,"['DT', 'NN', 'IN', 'DT', 'NNS', 'PRP', 'VBP', ..."
1,1,Remember when Jim Cramer a month ago said on h...,remember when jim cramer a month ago said on h...,"['VB', 'WRB', 'NNP', 'NNP', 'DT', 'NN', 'RB', ..."
2,2,"The FDIC only insures up to $250k, what happen...",the fdic only insures up to k what happens to ...,"['DT', 'NN', 'RB', 'VBZ', 'RP', 'TO', 'NN', 'W..."
3,3,"This is a pretty big deal , they had a huge am...",this is a pretty big deal they had a huge amou...,"['DT', 'VBZ', 'DT', 'RB', 'JJ', 'NN', 'PRP', '..."
4,4,If anyone's wondering how a bank with 200+ bil...,if anyones wondering how a bank with billion d...,"['IN', 'PRP', 'VBG', 'WRB', 'DT', 'NN', 'IN', ..."


In [10]:
labels = []
pattern = r"\b(" + "|".join(bad_words_list) + r")\b"
regex = re.compile(pattern)

for sentence in tqdm(news_dataset['cleaned_comment']):
    match = regex.findall(sentence.lower())
    labels.append(1) if match else labels.append(0)

100%|██████████| 63374/63374 [00:11<00:00, 5371.21it/s]


In [11]:
news_dataset['label'] = labels

In [12]:
news_dataset.to_csv('/Project/data/pos_tags_labels_dataset.csv',index=False)

In [13]:
bad_sentences = news_dataset[news_dataset['label']==1]
good_sentences = news_dataset[news_dataset['label']==0]

In [14]:
# good_sentences.to_csv('/Project/data/good_sentences.csv')
good_sentences.to_csv('/Project/data/good_sentences.csv')
bad_sentences.to_csv('/Project/data/bad_sentences.csv')

In [15]:
bad_sent = bad_sentences[['cleaned_comment','label', 'pos_tag']]
good_sent = good_sentences[['cleaned_comment','label', 'pos_tag']]

In [16]:
bad_comments = []
good_comments = []
for bad_rows in bad_sent.itertuples(index = True):
    bad_comments.append(bad_rows[1])

for good_rows in good_sent.itertuples(index = True):
    good_comments.append(good_rows[1])


Delete indexdir in data before running this

In [17]:
from org.apache.lucene.analysis.standard import StandardAnalyzer
from org.apache.lucene.analysis.en import EnglishAnalyzer
from org.apache.lucene.document import Document, Field, StringField, TextField
from org.apache.lucene.index import IndexWriter, IndexWriterConfig, DirectoryReader
from org.apache.lucene.search import IndexSearcher,ScoreDoc, TopDocs
from org.apache.lucene.queries.mlt import MoreLikeThisQuery
from org.apache.lucene.store import FSDirectory
from org.apache.lucene.util import Version
import java
import os

# Indexing
index_dir = '/Project/data/indexdir'
# analyzer = StandardAnalyzer()
analyzer = EnglishAnalyzer()

directory = FSDirectory.open(java.nio.file.Paths.get(index_dir))
config = IndexWriterConfig(analyzer)
writer = IndexWriter(directory, config)

if not os.path.exists(index_dir):
    print('Created dir')
    os.mkdir(index_dir)
    doc1 = Document()
    doc1.add(TextField("content", "The quick brown fox jumped over the lazy dog",  Field.Store.YES))
    doc1.add(TextField("pos_tag", "DT NN VB",  Field.Store.YES))
    writer.addDocument(doc1)
    
else:
    # Insert all good sentences to segments
    for index,good_sentence in good_sentences.iterrows():
        doc = Document()
        doc.add(TextField("content", good_sentence['cleaned_comment'], Field.Store.YES))
        doc.add(TextField("pos_tag", good_sentence['pos_tag'],  Field.Store.YES))
        writer.addDocument(doc)

# writer.commit()
writer.close()


In [18]:
from tqdm import tqdm
# Searching
searcher = IndexSearcher(DirectoryReader.open(directory))
analyzer = EnglishAnalyzer()
final_similar_sentences = []
final_pos_tags = []
# Get MoreLikeThisQuery for the first document
# more_like_this = MoreLikeThisQuery("dude my employer uses svb we are dead in the water right now",["content"],analyzer,'content')
for bad_sentence in tqdm(bad_sentences['cleaned_comment']):
    # print(f' BS: {bad_sentence}')
    similar_sentences=[]
    pos_tags_similar_sentences=[]
    pos_tags_bad_sentences=[]
    more_like_this = MoreLikeThisQuery(bad_sentence,["content"],analyzer,"content")
    hits = searcher.search(more_like_this,10).scoreDocs
    # print results
    # print(f"Found {len(hits)} hits:")
    for hit in hits:
        doc = searcher.doc(hit.doc)
        # print(f"{doc['content']}   score: {hit.score}")
        similar_sentences.append(doc['content'])
        pos_tags_similar_sentences.append(doc['pos_tag'])
    final_similar_sentences.append(similar_sentences)
    final_pos_tags.append(pos_tags_similar_sentences)





100%|██████████| 16764/16764 [00:13<00:00, 1212.30it/s]


In [19]:
bad_sentences_similar = pd.DataFrame({'bad_sentences':bad_sentences['cleaned_comment'],'bad_sentences_pos_tags':bad_sentences['pos_tag'],'similar_sentences':final_similar_sentences,'pos_tags_similar_sentences':final_pos_tags})

In [20]:
bad_sentences_similar.to_csv('/Project/data/bad_sentences_similar_pos_tags.csv',index=False)

In [21]:
final_similar_sentences[400]

['summer in canada is just less snow',
 'isnt dozens it is dozen',
 'i think i know which city will become the top destination this summer',
 'with a group of a dozen strangers less than three hours to decide lunch is pretty impressive',
 'there are dozens of us',
 'will probably continue wearing tops yeah but the chance at least increases besides some men are less picky about the',
 'ive known the law here in austin allows women to go top less i saw a woman at a park',
 'this is the biggest part of the problem',
 'hope it lasts until summer',
 'the biggest gang in la is the lapd']

In [23]:
bad_sentences['cleaned_comment'].iloc[400]

'my biggest issue with our top less women in summer is that they look like they have a dozen tits'